In [2]:
import requests
import json

def make_rmp_query(payload):
    # URL for the GraphQL endpoint
    url = 'https://www.ratemyprofessors.com/graphql'

    # Headers for the request
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Basic dGVzdDp0ZXN0',
    }

    # The payload for the GraphQL query
    # Convert the payload to JSON
    json_payload = json.dumps(payload)

    # Making the POST request
    response = requests.post(url, headers=headers, data=json_payload, timeout=10)
    return response.json()


In [3]:
all_teachers = {}
omitted_teachers = []

hasNextPage = True
end_cursor = ''

while hasNextPage:
    payload = {
      "query": "query TeacherSearchPaginationQuery(\n  $count: Int!\n  $cursor: String\n  $query: TeacherSearchQuery!\n) {\n  search: newSearch {\n    ...TeacherSearchPagination_search_1jWD3d\n  }\n}\n\nfragment TeacherSearchPagination_search_1jWD3d on newSearch {\n  teachers(query: $query, first: $count, after: $cursor) {\n    didFallback\n    edges {\n      cursor\n      node {\n        ...TeacherCard_teacher\n        id\n        __typename\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    resultCount\n    filters {\n      field\n      options {\n        value\n        id\n      }\n    }\n  }\n}\n\nfragment TeacherCard_teacher on Teacher {\n  id\n  legacyId\n  avgRating\n  numRatings\n  ...CardFeedback_teacher\n  ...CardSchool_teacher\n  ...CardName_teacher\n  ...TeacherBookmark_teacher\n}\n\nfragment CardFeedback_teacher on Teacher {\n  wouldTakeAgainPercent\n  avgDifficulty\n}\n\nfragment CardSchool_teacher on Teacher {\n  department\n  school {\n    name\n    id\n  }\n}\n\nfragment CardName_teacher on Teacher {\n  firstName\n  lastName\n}\n\nfragment TeacherBookmark_teacher on Teacher {\n  id\n  isSaved\n}\n",
      "variables": {
        "count": 1000,
        "cursor": end_cursor,
        "query": {
          "text": "",
          "schoolID": "U2Nob29sLTg4MQ==",
          "fallback": True
        }
      }
    }
    data = make_rmp_query(payload)

    print(len(data['data']['search']['teachers']['edges']))
    for teacher in data['data']['search']['teachers']['edges']:
        teacher = teacher['node']
        del teacher['__typename']
        teacher_name = f"{teacher['firstName']} {teacher['lastName']}"
        if teacher_name not in all_teachers:
            all_teachers[teacher_name] = teacher
        else:
            existing = all_teachers[teacher_name]
            if teacher['numRatings'] > existing['numRatings']:
                omitted_teachers.append(existing)
                all_teachers[teacher_name] = teacher
            else:
                omitted_teachers.append(teacher)

    end_cursor = data['data']['search']['teachers']['pageInfo']['endCursor']
    hasNextPage = data['data']['search']['teachers']['pageInfo']['hasNextPage']

1000
1000
1000
1000
682


In [4]:
len(all_teachers)

4596

In [5]:
len(omitted_teachers)

86

In [6]:
omitted_teacher_dict = {}
for teacher in omitted_teachers:
    teacher_name = f"{teacher['firstName']} {teacher['lastName']}"
    omitted_teacher_dict[teacher_name] = teacher

In [7]:
from datetime import datetime

# Get today's date in 'yyyymmdd' format
today_str = datetime.now().strftime('%Y%m%d')
today_str

'20241107'

In [8]:
with open(f'teacher_data/{today_str}_all_teachers_current.json', 'w') as f:
    json.dump(all_teachers, f)

In [9]:
with open(f'teacher_data/{today_str}_omitted_teachers.json', 'w') as f:
    json.dump(omitted_teacher_dict, f)